In [110]:
Description = """
Calculate the ring heater and CO2 settings in order to achieve maximum PRC gain in a cold state with the
substrate lens in a non-nominal state.  This is another way to show that the contrast defect is good or bad
in full lock and whether it maches our current estimates.

Strategy:  In the cold state, the PRC has the mode equal to the static lens with effectively no absorption
on the HR surface.  What we need to figure out is how to tune the ring heaters to get higher power recycling gain
in the cold state, this is a static value which will be additive to the amount of absorption we need.

Optical set up:
- ITMX ring heater at .28 Watts to counter the substrate lens difference and the radii of curvature difference
in the simple michelson.
- Substrate thermal lens is not nominal because there is no CO2 and no higher power.

09/10/2018
TVo
"""

In [123]:
import numpy as np
import pykat
import matplotlib.pyplot as pl
import trace_mode

%matplotlib inline

In [189]:
#### Find the amount of ring heater or annular heating required for the proper substrate lens
ITMXstat = -1./310812.+1./664100.
ITMYstat = 1.7E-6-1./1392000.

#absorption estimates for HR surfaces of TM .5ppm
abs_ix=0.2e-6 
abs_ex=0.2e-6 
abs_iy=0.483e-6
abs_ey=0.2e-6

#Power in the arms
P_opt = P_in =  50.0
prc_gain = 30
arm_gain = 250
P_arm = P_in*prc_gain*arm_gain 

# Paramters work out the TCS settings for O2
RH_SUBdef = -9e-6
#CO2_SUBdef = 6.23e-5 Old
CO2_SUBdef = 2.5e-5
SelfSUBdef = 4.87e-4

# Parameters for surface deformation 
RH_SURFdef = 9.91e-7
SelfSURFdef = -3.60e-5
    
## For a given absoprtion and static lens, return surface and substrate deformation
nomLens = 1/50e3

#Baseline arm cavity measurements
Lambda = 1064e-9

R_ix= 1940.3
R_ex= 2244.2
R_iy= 1939.2
R_ey= 2246.9

L = 3994.5

In [557]:
#Calculate Eigenmode
def cavity_info(Lambda, L, Rc1, Rc2):
    """
    Function computes several cavity paramaters
    Input: Lambda, L (length), Rc1, Rc2 (radii of curvature of cavity mirrors)
    Output: zr (Rayleigh range), w0 (waist size), z1 (distance waist<->mirror1)
            w1, w2 (spot size on both mirrors)
    """
    g1 = 1-L/Rc1
    g2 = 1-L/Rc2
    G  = g1*g2
    G1 = (g1+g2-2*G)

    k = 2.0*np.pi/Lambda

    zr = np.sqrt( L**2 * G * (1.0-G)/G1**2) 
    z1 = L * g2 * (1-g1) / G1
    z2 = L * g1 * (1-g2) / G1
    w0 =  np.sqrt( zr * Lambda / np.pi ) 
    w1 = np.sqrt( L * Lambda /np.pi * np.sqrt( g2 / (g1 * (1-G)) ))
    w2 = np.sqrt( L * Lambda /np.pi * np.sqrt( g1 / (g2 * (1-G)) ))
    q = z1+1j*zr
    return [zr, w0, z1, w1, w2, q]


def overlap(P_rh_ix,P_rh_iy,P_rh_ex,P_rh_ey,q_prc):

    dS_rh_ix = P_rh_ix * RH_SURFdef
    dS_rh_iy = P_rh_iy * RH_SURFdef
    dS_rh_ex = P_rh_ex * RH_SURFdef
    dS_rh_ey = P_rh_ey * RH_SURFdef

    R_ix_w_abs_rh = (1./R_ix + dS_rh_ix)**(-1.)
    R_ex_w_abs_rh = (1./R_ex + dS_rh_ex)**(-1.)
    R_iy_w_abs_rh = (1./R_iy + dS_rh_iy)**(-1.)
    R_ey_w_abs_rh = (1./R_ey + dS_rh_ey)**(-1.)

    xarm_w_abs_rh = cavity_info(Lambda, L, R_ix_w_abs_rh,R_ex_w_abs_rh)
    yarm_w_abs_rh = cavity_info(Lambda, L, R_ix_w_abs_rh,R_ey_w_abs_rh)
    
    overlap_prc = pykat.BeamParam.overlap(xarm_w_abs_rh[5], q_prc)
    overlap_arm = pykat.BeamParam.overlap(xarm_w_abs_rh[5], yarm_w_abs_rh[5])
    return [overlap_arm,overlap_prc]

In [545]:
### Calculating the RH power needed for cold lens
P_rh_ix = 2. * ( ITMXstat - ITMYstat + 1./R_iy - 1./R_ix) / (2*RH_SUBdef + RH_SURFdef)

print("The amount of RH power on ITMX is " +str(P_rh))

### Calculate the RH power for ETM mismatch
P_rh_ey = (-1./R_ey + 1./R_ex)/RH_SURFdef
print(P_rh_ey)

The amount of RH power on ITMX is 0.28230256317809543
0.5403122417784757


In [549]:
### Single pass actuation is therefore:
P_rh_ix_stat = P_rh_ix
P_rh_iy_stat = 0.0
P_rh_ex_stat = 0.0
P_rh_ey_stat = P_rh_ey

R_ix_w_rh=-( (1.0/R_ix) + (P_rh_ix_stat * RH_SURFdef) )**(-1.)
R_iy_w_rh=-( (1.0/R_iy) + (P_rh_iy_stat * RH_SURFdef) )**(-1.)
R_ex_w_rh= ( (1.0/R_ex) + (P_rh_ex_stat * RH_SURFdef) )**(-1.)
R_ey_w_rh= ( (1.0/R_ey) + (P_rh_ey_stat * RH_SURFdef) )**(-1.)

f_ix_w_rh = ( (ITMXstat) - (P_rh_ix * RH_SUBdef) )**(-1.)
f_iy_w_rh = ( (ITMYstat) - (P_rh_iy * RH_SUBdef) )**(-1.)

print(R_ix_w_rh,R_iy_w_rh,R_ex_w_rh,R_ey_w_rh)

-1939.247334080716 -1939.2000000000003 2244.2 2244.2


In [550]:
### Calculate the PRC Mode with the original lensing
katfile = "../kat_files/aLIGO_IFO_AWC_tuning_DCoff_maxtem2.kat"
basekat = pykat.finesse.kat() 
basekat.load(katfile)
basekat.parse("noxaxis")
basekat.parse("maxtem "+str(2))
kat=basekat.deepcopy() #copy kat code
kat.verbose = False

kat.ITMXHR.Rcx.value = R_ix
kat.ITMXHR.Rcy.value = R_ix

kat.ITMYHR.Rcx.value = R_iy
kat.ITMYHR.Rcy.value = R_iy

kat.ETMXHR.Rcx.value = R_ex
kat.ETMXHR.Rcy.value = R_ex

kat.ETMYHR.Rcx.value = R_ey
kat.ETMYHR.Rcy.value = R_ey

kat.ITMXTL.f = 1.0/ITMXstat
kat.ITMYTL.f = 1.0/ITMYstat

[q_PRX_ITMX_x,q_PRX_ITMX_y]  = trace_mode.from_cav_to_node(kat,'cavPRX','nITMX2')
[q_PRY_ITMY_x,q_PRY_ITMY_y]  = trace_mode.from_cav_to_node(kat,'cavPRY','nITMY2')

# Check that the PRC is modematched to itself with the orginal lenses, idiot check
overlap = pykat.BeamParam.overlap(q_PRX_ITMX_x,q_PRY_ITMY_x)
print(str((1-overlap)*1e6) +"ppm is the loss between PRX and PRY")


74.67575649622304ppm is the loss between PRX and PRY


In [551]:
### Calculate the PRC Mode with the new lensing

###We need to calculate the PRC mode with new substrate lens and HR surface. It's easiest to do this with Pykat
katfile = "../kat_files/aLIGO_IFO_AWC_tuning_DCoff_maxtem2.kat"
basekat = pykat.finesse.kat() 
basekat.load(katfile)
basekat.parse("noxaxis")
basekat.parse("maxtem "+str(2))
kat=basekat.deepcopy() #copy kat code
kat.verbose = False

kat.ITMXHR.Rcx.value = R_ix_w_rh
kat.ITMXHR.Rcy.value = R_ix_w_rh

kat.ITMYHR.Rcx.value = R_iy_w_rh
kat.ITMYHR.Rcy.value = R_iy_w_rh

kat.ETMXHR.Rcx.value = R_ex_w_rh
kat.ETMXHR.Rcy.value = R_ex_w_rh

kat.ETMYHR.Rcx.value = R_ey_w_rh
kat.ETMYHR.Rcy.value = R_ey_w_rh

kat.ITMXTL.f = -f_ix_w_rh
kat.ITMYTL.f = -f_iy_w_rh

[q_PRX_ITMX_x,q_PRX_ITMX_y]  = trace_mode.from_cav_to_node(kat,'cavPRX','nITMX2')
[q_PRY_ITMY_x,q_PRY_ITMY_y]  = trace_mode.from_cav_to_node(kat,'cavPRY','nITMY2')

# Check that the PRC is modematched to itself with the new radii of curvature
overlap = pykat.BeamParam.overlap(q_PRX_ITMX_x,q_PRY_ITMY_x)
print(str((1-overlap)*1e6) +"ppm is the loss between PRX and PRY")

1.5417705959697159ppm is the loss between PRX and PRY


In [558]:
overlap(0.0,0.0,0.0,0.0,-np.conjugate(q_PRX_ITMX_x))

[0.9999846841125842, 0.9841114618944211]

In [559]:
### After correcting for static mode mismatch with ITMX RH, the arms are 
### relatively well matched, but the prc to arms is really bad.

[arm2arm,arm2prc] = overlap(P_rh_ix_stat,0.0,0.0,P_rh_ey_stat,-np.conjugate(q_PRX_ITMX_x))

print(str((1-arm2prc)*1e6), "ppm is the loss between the prc and the arms")
print(str((1-arm2arm)*1e6), "ppm is the loss between the xarm and yarm")

15642.3273393671 ppm is the loss between the prc and the arms
0.0 ppm is the loss between the xarm and yarm


In [560]:
### Calculate the static ETM mismatch

1.0

In [569]:
### We can search for the proper modematching between the arms in the ETMS first

P_rh_ex = np.arange(0,10.0,.01)
P_rh_ey = np.arange(0,10.0,.01)

X, Y = np.meshgrid(P_rh_ex, P_rh_ey)

overlap_endRH_on = overlap(P_rh_ix_stat,0.0,X,P_rh_ey_stat+Y,-np.conjugate(q_PRX_ITMX_x))

In [573]:
overlap_endRH_on[0]

array([[1.        , 0.99999999, 0.99999998, ..., 0.99341266, 0.99339785,
        0.99338302],
       [0.99999999, 1.        , 0.99999999, ..., 0.99342446, 0.99340966,
        0.99339484],
       [0.99999998, 0.99999999, 1.        , ..., 0.99343624, 0.99342146,
        0.99340666],
       ...,
       [0.99341266, 0.99342446, 0.99343624, ..., 1.        , 0.99999999,
        0.99999997],
       [0.99339785, 0.99340966, 0.99342146, ..., 0.99999999, 1.        ,
        0.99999999],
       [0.99338302, 0.99339484, 0.99340666, ..., 0.99999997, 0.99999999,
        1.        ]])

In [574]:
Z_arm = overlap_endRH_on[1]

# Find the indices which have good mode matching between the arms
indices = np.argwhere(Z_arm<1)

#Powers relating to good mode matching between the arms
ex_p = []
ey_p= []
for i in range(len(indices)):
    xs = float(X[int(indices[i][0])][int(indices[i][1])])
    ex_p.append(xs)
    ys = float(Y[int(indices[i][0])][int(indices[i][1])])
    ey_p.append(ys)

ex_p=np.asarray(ex_p)
ey_p=np.asarray(ey_p)

In [571]:
print(ex_p,ey_p)

[0.   0.01 0.02 ... 9.97 9.98 9.99] [0.   0.   0.   ... 9.99 9.99 9.99]


In [575]:
# We know with the previous list that the arms are well modematched, so we need to calculate the prc overlap
prc_overlap_rh = (1.0-overlap(P_rh,0.0,ex_p,ey_p,-np.conjugate(q_PRX_ITMX_x))[1])*1e6
print(overlap(P_rh,0.0,ex_p,ey_p,-np.conjugate(q_PRX_ITMX_x))[1])

[0.98435767 0.98437569 0.98439369 ... 0.99803244 0.99804056 0.99804867]


In [576]:
opt_mm_ind = prc_overlap_rh.argmin()
opt_mm_ex_pwr = ex_p[opt_mm_ind]
opt_mm_ey_pwr = ey_p[opt_mm_ind]

print("Optimal EX RH Power "+ str(opt_mm_ex_pwr))
print("Optimal EY RH Power "+ str(opt_mm_ey_pwr))
print("This will give a mode mismatch of " +str(round(prc_overlap_rh.min(),6)) + " PPM between the arms and PRC")

Optimal EX RH Power 9.99
Optimal EY RH Power 0.0
This will give a mode mismatch of 1951.333502 PPM between the arms and PRC
